In [ ]:
import qiskit 
from mitiq.pec.representations.depolarizing import represent_operation_with_global_depolarizing_noise

qreg = qiskit.QuantumRegister(2, name="test_name")
cnot_circuit = qiskit.QuantumCircuit(qreg)
cnot_circuit.cnot(0, 1)

rep = represent_operation_with_global_depolarizing_noise(cnot_circuit, 0.1)
print(rep)

In [ ]:
from mitiq.interface import convert_to_mitiq, convert_from_mitiq
converted, in_type = convert_to_mitiq(cnot_circuit)
print(converted)

In [ ]:
from typing import List
from cirq import Operation, X, Y, Z, Circuit
post_ops: List[List[Operation]]
qubits = converted.all_qubits()
print(qubits)

In [ ]:
from cirq import X, Y, Z, Circuit
noise_level = 0.1
# The single-qubit case: linear combination of 1Q Paulis
if len(qubits) == 1:
    q = tuple(qubits)[0]

    epsilon = 4 / 3 * noise_level
    alpha_pos = 1 + ((3 / 4) * epsilon / (1 - epsilon))
    alpha_neg = -(1 / 4) * epsilon / (1 - epsilon)

    alphas = [alpha_pos] + 3 * [alpha_neg]
    post_ops = [[]]  # for alpha_pos, we do nothing, rather than I
    post_ops += [[P(q)] for P in [X, Y, Z]]  # 1Q Paulis

# The two-qubit case: linear combination of 2Q Paulis
elif len(qubits) == 2:
    q0, q1 = qubits

    epsilon = 16 / 15 * noise_level
    alpha_pos = 1 + ((15 / 16) * epsilon / (1 - epsilon))
    alpha_neg = -(1 / 16) * epsilon / (1 - epsilon)

    alphas = [alpha_pos] + 15 * [alpha_neg]
    post_ops = [[]]  # for alpha_pos, we do nothing, rather than I x I
    post_ops += [[P(q0)] for P in [X, Y, Z]]  # 1Q Paulis for q0
    post_ops += [[P(q1)] for P in [X, Y, Z]]  # 1Q Paulis for q1
    post_ops += [
        [Pi(q0), Pj(q1)] for Pi in [X, Y, Z] for Pj in [X, Y, Z]
    ]  # 2Q Paulis

# Basis of implementable operations as circuits.
imp_op_circuits = [converted + Circuit(op) for op in post_ops]
print(imp_op_circuits)

In [ ]:
# Convert back to input type.
circuits = [convert_from_mitiq(c, in_type) for c in imp_op_circuits]
print(circuits)

In [ ]:
# Build basis expansion.
from mitiq.pec.types import NoisyOperation
expansion = {NoisyOperation(c): a for c, a in zip(circuits, alphas)}
print(expansion)

In [ ]:
from mitiq.pec.types import OperationRepresentation
representation = OperationRepresentation(cnot_circuit, expansion)
print(representation)